In [1]:
# ОПИСАНИЕ ЗАДАНИЯ
#Построй линейный прогноз любого из интересующего тебя пункта отчетности.
#Файлы которые я тебе скину - это реальная отчетность с интерфакса из под нашей OCR.
#Поэтому заодно применишь уже полученные знания в обработке плохо собранных таблиц
#Тебе нужно будет вытащить нужные ячейки, построить из этого график и спрогнозировать куда будет двигаться
#выбранный тобой пункт отчетности с последующими отчетностями.
#Как раз отработаешь уже изученные обработки таблиц и данных, построение графиков,
#методы группировки и скорее всего будешь вынуждена поработать с numpy, если не трогала его еще, как раз познакомишься.
#Ну и конечно задача регрессии машинного обучения что у тебя фактически актуальной темой является

In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import re

In [3]:
def add_balance(data):
    '''функция для добавления новых столбцов в df_balance'''
    data = data.dropna(how='all')
    for item in data.index:
        if data.iloc[item, 0] == 'Наименование показателя':
            i = item
            break
    new_column_list = data.iloc[i, 2:5].dropna().to_list()
    for item in data.index:
        if data.iloc[item, 1] == '1700':
            j = item
            break
    balance = data.iloc[(i+1):(j+1), 2:(2+len(new_column_list))]
    balance = balance.reset_index(drop=True)
    balance.columns = new_column_list
    return balance


def add_assets(data_new):
    '''функция для добавления активов из годовых балансов'''
    for column in data_new.columns:
        for item in data_new.index:
            if re.search('Внеоборотные активы Нематериальные активы', str(data_new.iloc[item, column])) != None:
                i = item
                c = column
                break
    new_column_list = data_new.iloc[(i-2), c:(c+4)].to_list()
    for column in data_new.columns:
        for item in data_new.index:
            if re.search('сумма строк 1100', str(data_new.iloc[item, column])) != None:
                j = item
                break
    balance1 = data_new.iloc[i:(j), c:(c+4)]
    balance1 = balance1.reset_index(drop=True)
    balance1.columns = new_column_list
    balance1 = balance1.dropna()
    return balance1


def add_liabilities(data_new):
    '''функция для добавления пассивов из годовых балансов'''
    for column in data_new.columns:
        for item in data_new.index:
            if re.search('Капитал и резервы Уставный капитал', str(data_new.iloc[item, column])) != None:
                m = item
                n = column
                break
    new_column_list = data_new.iloc[(m-2), n:(n+4)].to_list()
    for item in data_new.index:
        if re.search('сумма строк 1300', str(data_new.iloc[item, n])) != None:
            j = item
            break
    balance2 = data_new.iloc[(m):(j+1), n:(n+4)]
    balance2 = balance2.reset_index(drop=True)
    balance2.columns = new_column_list
    balance2 = balance2.dropna()
    return balance2

In [4]:
mypath = r'C:\Users\singh_000\MyProjects\Statements_project\statements\\'
files_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
df_balance = pd.DataFrame()
for file in files_list:
    data_file = pd.ExcelFile(mypath+file)
    data = pd.read_excel(data_file, header=None)
    balance = add_balance(data)
    df_balance[balance.columns] = balance

In [6]:
df_balance = df_balance.dropna(axis=1)

In [7]:
df_balance.columns = ['2020.06.30', '2019.12.31', '2020.03.31', '2019.09.30', '2017.12.31', '2018.06.30',
                      '2016.12.31', '2017.09.30', '2015.12.31', '2016.09.30',
                      '2015.12.31', '2014.12.31', '2016.06.30', '2016.03.31',
                      '2015.06.30', '2013.12.31', '2014.06.30', '2012.12.31']

In [8]:
df_balance = df_balance.loc[:, ~df_balance.columns.duplicated()]

In [9]:
df_balance = df_balance.reindex(sorted(df_balance.columns), axis=1)

In [10]:
data = pd.read_excel("./712_2509_0-Copy1.xlsx", header=None)
data = data.dropna(how='all')
for item in data.index:
    if data.iloc[item, 0] == 'Наименование показателя':
        i = item
        break
    for item in data.index:
        if data.iloc[item, 1] == '1700':
            j = item
            break
text_column_list = data.iloc[i, 0:2].dropna().to_list()
data = data.iloc[(i+1):(j+1), 0:2]
data = data.reset_index(drop=True)
data.columns = text_column_list
data[df_balance.columns] = df_balance

In [11]:
df_data = data.drop(0)

In [12]:
df_data.reset_index(drop=True, inplace=True)

In [13]:
mypath = r'C:\Users\singh_000\MyProjects\Statements_project\statements1\\'
files_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [14]:
df_balance1 = pd.DataFrame()
for file in files_list:
    data_file = pd.ExcelFile(mypath+file)
    data = pd.read_excel(data_file, header=None)
    balance_new = add_assets(data)
    df_balance1[balance_new.columns] = balance_new

In [15]:
df_balance2 = pd.DataFrame()
for file in files_list:
    data_file = pd.ExcelFile(mypath+file)
    data = pd.read_excel(data_file, header=None)
    balance_new = add_liabilities(data)
    df_balance2[balance_new.columns] = balance_new

In [16]:
balance_final = df_balance1.append(df_balance2, ignore_index=True)
balance_f = balance_final.drop(['Наименование показателя', 'Код'], axis=1)
balance_f.columns

In [19]:
balance_f.columns = ['2017.12.31', '2016.12.31', '2015.12.31', '2014.12.31', '2013.12.31',
                     '2012.12.31', '2011.12.31', '2010.12.31']

In [20]:
data_final = pd.concat([df_data, balance_f], axis=1)
data_final = data_final.loc[:, ~data_final.columns.duplicated()]
data_final = data_final.reindex(sorted(data_final.columns), axis=1)

In [23]:
data_final = data_final.loc[:, ['Наименование показателя', 'Код', '2010.12.31', '2011.12.31', '2012.12.31', '2013.12.31', '2014.06.30',
                                '2014.12.31', '2015.06.30', '2015.12.31', '2016.03.31', '2016.06.30',
                                '2016.09.30', '2016.12.31', '2017.09.30', '2017.12.31', '2018.06.30',
                                '2019.09.30', '2019.12.31', '2020.03.31', '2020.06.30']]

In [24]:
data_final = data_final.T
name_list = data_final.loc['Наименование показателя'].to_list()
data_final = data_final.drop('Наименование показателя')
data_final.columns = name_list
data_final.drop('Код', axis=0)